In [1]:
import pandas as pd
import numpy as np
import re
import requests
import datetime
import os
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import ambiance

In [2]:
def read_xls(file, sheet):
    
    try:
        df = pd.read_excel(file,sheet)
    except FileNotFoundError as e:
        print("Excel file not found " + str(e))
        sys.exit(1)
    
    return df

In [3]:
def read_csv(file):
    
    try:
        df = pd.read_csv(file, sep=' ')
    except FileNotFoundError as e:
        print("Excel file not found " + str(e))
        sys.exit(1)
    
    return df

In [4]:
def calculate_bearing(d):
  dirs = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
  ix = int(round(d / (360. / len(dirs))))
    
  return dirs[ix % len(dirs)]

In [5]:
def convert_lat_gps_coord(coor):
    
    lat_deg = coor[:2]
    lat_min = coor[2:4]
    lat_sec = coor[5:7]
    direction = coor[7]
    
    dd = (float(lat_deg) + float(lat_min)/60 + float(lat_sec)/(60*60)) * (-1 if direction in ['W', 'S'] else 1)

    return dd

In [6]:
def convert_lon_gps_coord(coor):
    
    lon_deg = coor[1:3]
    lon_min = coor[3:5]
    lon_sec = coor[6:8]
    direction = coor[8]
    
    dd = (float(lon_deg) + float(lon_min)/60 + float(lon_sec)/(60*60)) * (-1 if direction in ['W', 'S'] else 1)
    
    return dd

In [7]:
def plot(lat, lng, zoom=10, map_type='roadmap'):
    gmap_options = GMapOptions(lat=lat, lng=lng, 
                               map_type=map_type, zoom=zoom)
    p = gmap(api_key, gmap_options, title='Pays de Gex', 
             width=bokeh_width, height=bokeh_height)
    # definition of the column data source: 
    source = ColumnDataSource(df)
    # see how we specify the x and y columns as strings, 
    # and how to declare as a source the ColumnDataSource:
    center = p.circle('lon', 'lat', size=4, alpha=0.2, 
                      color='yellow', source=source)
    show(p)
    return p

In [8]:
def Alt_toPressure(val):

    return ambiance.Atmosphere(val).pressure[0]

In [9]:
def Alt_toTemp(val):
    
    return ambiance.Atmosphere.T2t(ambiance.Atmosphere(val).temperature)[0]

In [10]:
def Alt_toLayerName(val):
    
    return ambiance.Atmosphere(val).layer_name[0]

In [11]:
def Pressure_toAlt(val):

    return ambiance.Atmosphere.from_pressure(val).h[0]

In [12]:
def Pressure_toTemp(val):
    
    return ambiance.Atmosphere.T2t(ambiance.Atmosphere.from_pressure(val).temperature)[0]

In [13]:
def Clean_Pressure(val):

    # Remove bad data
    if val > 200000:
        return np.nan
    return val

### Read Input Telemetry Spreadsheet

In [14]:
df = read_xls("KB9LNS-14.xlsx","KB9LNS-14")

In [15]:
df.sample(10)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12
61,2023-04-21,10:12:25,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,N9XJN-5:/151219h4...",00124H,8.80C,2157hPa,0.00C,0.00hPa,0.00,NaN,NaN
121,2023-04-21,10:53:56,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,K9MQ-1:/155351h41...",00115H,4.10C,2141hPa,0.00C,0.00hPa,0.00,NaN,NaN
36,2023-04-21,09:54:09,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KX9A-1:/145403h40...",00125H,16.5C,293.hPa,0.00C,0.00hPa,0.00,NaN,NaN
32,2023-04-21,09:51:40,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAO,KS9A-10:>",5.79V,08S,00114H,00000000F,0090T,0073T,NaN,NaN
141,2023-04-21,11:19:18,CDT:,"KB9LNS-14>APLIGA,W9AZ-2,WIDE1*,WIDE2-1,qAR,K9M...",00104H,13.2C,684.hPa,0.00C,0.00hPa,0.00,NaN,NaN
31,2023-04-21,09:51:05,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAO,AB9JJ-10:/145059h...",00114H,17.7C,377.hPa,0.00C,0.00hPa,0.00,NaN,NaN
21,2023-04-21,09:43:01,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAO,KS9A-10:/144255h4...",00133H,21.1C,652.hPa,0.00C,0.00hPa,0.00,NaN,NaN
135,2023-04-21,11:06:39,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KX9A-1:>",5.61V,07S,00184H,00000000F,0179T,0147T,NaN,NaN
150,2023-04-21,11:34:30,CDT:,"KB9LNS-14>APLIGA,W9AZ-2,WIDE1*,WIDE2-1,qAR,K9M...",00110H,17.0C,847.hPa,0.00C,0.00hPa,0.00,NaN,NaN
113,2023-04-21,10:47:53,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KX9A-1:/154747h41...",00106H,5.40C,2148hPa,0.00C,0.00hPa,0.00,NaN,NaN


In [16]:
# Label columns
col_names = { 
    "Column1":"Date", 
    "Column2":"Time",
    "Column3":"TZ",
    "Column4":"Path",
    "Column5":"HDOP",
    "Column6":"Interior Temp (C)",
    "Column7":"Interior Pressure",
    "Column8":"Exterior Temp (C)",
    "Column9":"Exterior Pressure",
    "Column10":"Relative Humdity"
            }

In [17]:
df.rename(col_names, axis="columns", inplace=True)

In [18]:
df.columns

Index(['Date', 'Time', 'TZ', 'Path', 'HDOP', 'Interior Temp (C)',
       'Interior Pressure', 'Exterior Temp (C)', 'Exterior Pressure',
       'Relative Humdity', 'Column11', 'Column12'],
      dtype='object')

In [19]:
# View large sample of df
df.head(15)

,Date,Time,TZ,Path,HDOP,Interior Temp (C),Interior Pressure,Exterior Temp (C),Exterior Pressure,Relative Humdity,Column11,Column12
0,2023-04-21,09:26:49,CDT:,"KB9LNS-14>APLIGA,WIDE1-1,WIDE2-1,qAR,KC8RFE-3:...",00378H,23.3C,1095hPa,0.00C,0.00hPa,0.00,NaN,NaN
1,2023-04-21,09:27:49,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAO,KS9A-10:/142743h4...",00172H,23.3C,1075hPa,0.00C,0.00hPa,0.00,NaN,NaN
2,2023-04-21,09:28:49,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KC8RFE-3:/142843h...",00136H,23.3C,1048hPa,0.00C,0.00hPa,0.00,NaN,NaN
3,2023-04-21,09:29:49,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KC8RFE-3:/142943h...",00144H,23.3C,1026hPa,0.00C,0.00hPa,0.00,NaN,NaN
4,2023-04-21,09:30:49,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KC8RFE-3:/143043h...",00149H,23.8C,1029hPa,0.00C,0.00hPa,0.00,NaN,NaN
5,2023-04-21,09:31:24,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KX9A-1:>",5.79V,06S,00149H,00000000F,0066T,0053T,NaN,NaN
6,2023-04-21,09:31:25,CDT:,"KB9LNS-14>APLIGA,KS9A-10*,qAR,W9PFD:",5.79V,06S,00149H,00000000F,0066T,0053T,NaN,[Unsupported
7,2023-04-21,09:31:53,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KX9A-1:/143147h40...",00237H,23.8C,1006hPa,0.00C,0.00hPa,0.00,NaN,NaN
8,2023-04-21,09:32:53,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAO,KS9A-10:/143247h4...",00149H,23.6C,971.hPa,0.00C,0.00hPa,0.00,NaN,NaN
9,2023-04-21,09:33:53,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KC8RFE-3:/143347h...",00111H,23.3C,932.hPa,0.00C,0.00hPa,0.00,NaN,NaN


### Prepare Telemetry Tracker Data

In [20]:
# Filter out telemetry from status messages (status message contain battery level)

df1 = pd.DataFrame(df[ df['HDOP'].str.endswith('V') == True ])
df2 = pd.DataFrame(df[ df['HDOP'].str.endswith('V') == False ])

In [21]:
# Tracker status info
df1.head(5)

,Date,Time,TZ,Path,HDOP,Interior Temp (C),Interior Pressure,Exterior Temp (C),Exterior Pressure,Relative Humdity,Column11,Column12
5,2023-04-21,09:31:24,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KX9A-1:>",5.79V,06S,00149H,00000000F,0066T,0053T,NaN,NaN
6,2023-04-21,09:31:25,CDT:,"KB9LNS-14>APLIGA,KS9A-10*,qAR,W9PFD:",5.79V,06S,00149H,00000000F,0066T,0053T,NaN,[Unsupported
12,2023-04-21,09:36:28,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KC8RFE-2:>",5.79V,06S,00139H,00000000F,0072T,0058T,NaN,NaN
18,2023-04-21,09:41:32,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KX9A-1:>",5.79V,07S,00141H,00000000F,0078T,0063T,NaN,NaN
19,2023-04-21,09:41:32,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,W9PFD:",5.79V,07S,00141H,00000000F,0078T,0063T,NaN,[Unsupported


In [22]:
# Flight telemetry info
df2.head(5)

,Date,Time,TZ,Path,HDOP,Interior Temp (C),Interior Pressure,Exterior Temp (C),Exterior Pressure,Relative Humdity,Column11,Column12
0,2023-04-21,09:26:49,CDT:,"KB9LNS-14>APLIGA,WIDE1-1,WIDE2-1,qAR,KC8RFE-3:...",00378H,23.3C,1095hPa,0.00C,0.00hPa,0.00,NaN,NaN
1,2023-04-21,09:27:49,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAO,KS9A-10:/142743h4...",00172H,23.3C,1075hPa,0.00C,0.00hPa,0.00,NaN,NaN
2,2023-04-21,09:28:49,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KC8RFE-3:/142843h...",00136H,23.3C,1048hPa,0.00C,0.00hPa,0.00,NaN,NaN
3,2023-04-21,09:29:49,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KC8RFE-3:/142943h...",00144H,23.3C,1026hPa,0.00C,0.00hPa,0.00,NaN,NaN
4,2023-04-21,09:30:49,CDT:,"KB9LNS-14>APLIGA,WIDE2-1,qAR,KC8RFE-3:/143043h...",00149H,23.8C,1029hPa,0.00C,0.00hPa,0.00,NaN,NaN


In [23]:
# Expand Telemetry data into all fields

# APRS Telemetry field
df2['Telemetry'] = df2['Path'].str.split(':', expand=True)[1]
df2['Path']      = df2['Path'].str.split(':', expand=True)[0]

# GPS Time and Location
df2['GPS Time']      = df2['Telemetry'].str.split('/', expand=True)[1].str.split('h', expand=True)[0]
df2['Lat']           = df2['Telemetry'].str.split('/', expand=True)[1].str.split('h', expand=True)[1]
df2['Lon']           = df2['Telemetry'].str.split('/', expand=True)[2].str.split('W', expand=True)[0] + 'W'
df2['Bearing']       = df2['Telemetry'].str.split('/', expand=True)[2].str.split('W', expand=True)[1]
df2['Speed (mph)']   = df2['Telemetry'].str.split('/', expand=True)[3].astype(float)
df2['Altitude (ft)'] = df2['Telemetry'].str.split('/', expand=True)[4].str.split('=', expand=True)[1].astype(float)

In [27]:
# Unit Conversions

# Time
df2['Time'] = pd.to_datetime(df2['GPS Time'].str[:6],format= '%H%M%S' ).dt.time

# Speed and distance
df2['Speed (kph)']  = df2['Speed (mph)'].astype(float) * 1.609344
df2['Altitude (m)'] = df2['Altitude (ft)'].astype(float) * 0.3048
df2['HDOP (ft)'] = df2['HDOP'].str[:5].astype(int)

# Tempuratures
df2['Interior Temp (F)'] = df2['Interior Temp (C)'].str.split('C', expand=True)[0].astype(float)*(9/5)+32
#df2['Exterior Temp (F)'] = df2['Exterior Temp (C)'].str.split('C', expand=True)[0].astype(float)*(9/5)+32

# Pressure
df2['I Pressure'] = df2['Interior Pressure'].str[:4].astype(float)
df2['I Pressure'] = pd.Series(df2['I Pressure']*100).apply(Clean_Pressure).ffill()
#df2['E Pressure'] = df2['Exterior Pressure'].str[:4].astype(float)

In [28]:
# Compute new telemetry data fields

# Ascent Rate (ft)
df2['Altitude Delta (ft)'] = df2['Altitude (ft)'].diff()
df2['Time Delta (s)'] = pd.to_timedelta(df2['Time'].astype(str)).diff(-1).dt.total_seconds().abs()
df2['Rate (fps)'] = df2['Altitude Delta (ft)'].astype(float) / df2['Time Delta (s)'].astype(float)

# Ascent Rate (m)
df2['Altitude Delta (m)'] = df2['Altitude (m)'].diff()
df2['Rate (mps)'] = df2['Altitude Delta (m)'].astype(float) / df2['Time Delta (s)'].astype(float)

# Direction
df2['Cardinal Direction'] = df2['Bearing'].str[1:].astype(float).apply(calculate_bearing)

# Time
df2['GPS Epoch'] = pd.to_timedelta(df2['Time'].astype(str)).dt.total_seconds().astype(int)
epoch=df2['GPS Epoch'].iloc[0]
df2['GPS Epoch'] = df2['GPS Epoch'] - epoch

#### Atmospheric Predictions

In [29]:
# Atmospheric predictions using ISA reference data

# Altitude based predictions
df2['Predict Alt Pressure'] = df2['Altitude (m)'].apply(Alt_toPressure)
df2['Predict Alt Temp(C)'] = df2['Altitude (m)'].apply(Alt_toTemp)
df2['Predict Alt Temp(F)'] = df2['Predict Temp(C)'].astype(float)*(9/5)+32

# Pressure based predictions
df2['Predict IP Alt(m)'] = pd.Series(df2['I Pressure']).apply(Pressure_toAlt)
df2['Predict IP Alt(ft)'] = pd.Series(df2['I Pressure']).apply(Pressure_toAlt) * 3.2808399
df2['Predict IP Temp(C)'] = pd.Series(df2['I Pressure']).apply(Pressure_toTemp)
df2['Predict IP Temp(F)'] = df2['Predict IP Temp(C)']*(9/5)+32

# Pressure based predictions
#df2['Predict EP Alt(m)'] = pd.Series(df2['E Pressure']*100).apply(Pressure_toAlt)
#df2['Predict EP Alt(ft)'] = df2['Predict EP Alt(m)'] * 3.2808399
#df2['Predict EP Temp(C)'] = pd.Series(df2['E Pressure']*100).apply(Pressure_toTemp)
#df2['Predict EP Temp(F)'] = df2['Predict EP Temp(C)']*(9/5)+32

KeyError: 'Predict Temp(C)'

In [ ]:
df2.iloc[30]

In [ ]:
# Convert pressure from Pa to hPa
df2['I Pressure'] = df2['I Pressure']/100
df2['Predict Pressure'] = df2['Predict Pressure']/100

### Read International Standard Atmosphere Tables

In [ ]:
isa = read_csv("International Standard Atmosphere Table.txt")

In [ ]:
isa.head(10)

In [ ]:
isa['hpa'] = isa['(N/m2)']/100

In [ ]:
isa['(F)'] = (isa['(◦C)'].str.strip(' ').str.replace('−','-').astype(float)*(9/5))+32

In [ ]:
isa.sample(5)

In [ ]:
%matplotlib inline

plt.figure(dpi=600, figsize=(10,12))
plot1 = plt.subplot2grid((3,1),(0,0))
plot2 = plt.subplot2grid((3,1),(1,0))
plot3 = plt.subplot2grid((3,1),(2,0))

plot1.scatter('GPS Epoch', 'Altitude (ft)', data=df2, marker='.', color='green', cmap=plt.get_cmap("jet"), alpha=0.5, linewidth=0.3)
plot1.scatter('GPS Epoch', 'Predict IP Alt(ft)', data=df2, marker='.', color='blue', cmap=plt.get_cmap("jet"), alpha=0.3, linewidth=0.3)
plot1.tick_params(axis='both', which='both', reset=True, direction='inout', labelsize='xx-small', bottom=True, length=5, width=0.3, grid_color='grey', grid_alpha=0.5)
plot1.set_xlabel('Time (sec)', size='x-small')
plot1.set_ylabel('Altitude (feet)', size='x-small')
plot1.set_title('Altitude vs Time', color='blue', size='small', pad=10.0, weight='bold')
plot1.grid(visible=True, which='both', linewidth=0.3)
plot1.set_ylim([-10000,130000])
plot1.set_xlim([0,9000])
plot1.legend(['Altitude (ft) - GPS', 'Altitude (ft) - Predicted - Interior Pressure'], fontsize='x-small', loc='upper left')

plot2.scatter('GPS Epoch', 'Rate (mps)', data=df2, marker='.', color='orange', cmap=plt.get_cmap("jet"), alpha=0.5,linewidth=0.3)
plot2.tick_params(axis='both', which='both', reset=True, direction='inout', labelsize='xx-small', bottom=True, length=5, width=0.3, grid_color='grey', grid_alpha=0.5)
plot2.set_xlabel('Time (sec)', size='x-small')
plot2.set_ylabel('Speed (m/s)', size='x-small')
plot2.set_title('Ascent/Descent Rate', color='blue', size='small', pad=10.0, weight='bold')
plot2.grid(visible=True, which='both', linewidth=0.3)
plot2.set_ylim([-30,15])
plot2.set_xlim([0,9000])

plot3.scatter('GPS Epoch', 'Speed (mph)', data=df2, marker='.', color='brown', cmap=plt.get_cmap("jet"), alpha=0.5, linewidth=0.3)
plot3.tick_params(axis='both', which='both', reset=True, direction='inout', labelsize='xx-small', bottom=True, length=5, width=0.3, grid_color='grey', grid_alpha=0.5)
plot3.set_xlabel('Time (sec)', size='x-small')
plot3.set_ylabel('Speed (mph)', size='x-small')
plot3.set_title('Speed (mph)', color='blue', size='small', pad=10.0, weight='bold')
plot3.grid(visible=True, which='both', linewidth=0.3)
plot3.set_ylim([-10,80])
plot3.set_xlim([0,9000])

plt.tight_layout(pad=1.5)
plt.show()

In [ ]:
%matplotlib inline

plt.figure(dpi=300, figsize=(12,8))
plot4 = plt.subplot2grid((2,1),(0,0))
plot5 = plt.subplot2grid((2,1),(1,0))

plot4.scatter('Interior Temp (F)', 'Altitude (ft)', data=df2, marker='.', color='orange', cmap=plt.get_cmap("jet"), alpha=0.5, linewidth=0.3)
plot4.scatter('Predict Temp(F)', 'Altitude (ft)', data=df2, marker='.', color='blue', cmap=plt.get_cmap("jet"), alpha=0.3, linewidth=0.3)
plot4.tick_params(axis='both', which='both', reset=True, direction='inout', labelsize='xx-small', bottom=True, length=5, width=0.3, grid_color='grey', grid_alpha=0.5)
plot4.set_xlabel('Tempurature (F)', size='x-small')
plot4.set_ylabel('Altitude (ft)', size='x-small')
plot4.set_title('Temperature vs. Altitude', color='blue', size='small', pad=10.0, weight='bold')
plot4.grid(visible=True, which='both', linewidth=0.3)
plot4.set_xlim([-80,95])
plot4.set_ylim([-10000,110000])
plot4.legend(['Interior Temp', 'Predicted Temp', 'Exterior Temp'], fontsize='x-small', loc='lower left')

plot5.scatter('GPS Epoch', 'Interior Temp (F)', data=df2, marker='.', color='blue', cmap=plt.get_cmap("jet"), alpha=0.4, linewidth=0.3)
plot5.scatter('GPS Epoch', 'Predict Temp(F)', data=df2, marker='.', color='red', cmap=plt.get_cmap("jet"), alpha=0.5, linewidth=0.3)
plot5.tick_params(axis='both', which='both', reset=True, direction='inout', labelsize='xx-small', bottom=True, length=5, width=0.3, grid_color='grey', grid_alpha=0.5)
plot5.set_xlabel('Time (sec)', size='x-small')
plot5.set_ylabel('Payload Temp (F)', size='x-small')
plot5.set_title('Payload Temp (F)', color='blue', size='small', pad=10.0, weight='bold')
plot5.grid(visible=True, which='both', linewidth=0.3)
plot5.set_ylim([-80,95])
plot5.set_xlim([0,9000])
plot5.legend(['Interior Temp', 'Predicted Temp'], fontsize='x-small', loc='lower left')

plt.tight_layout(pad=1.5)
plt.show()

In [ ]:
df2.columns

In [ ]:
%matplotlib inline

plt.figure(dpi=175, figsize=(10,6))
plot1 = plt.subplot2grid((1,1),(0,0))

plot1.scatter('Predict Pressure', 'Altitude (ft)', marker='.', data=df2, color='red', alpha=0.4, linewidth=0.8)
plot1.scatter('I Pressure', 'Altitude (ft)', marker='.', data=df2, color='darkgreen', alpha=0.4, linewidth=0.8)
plot1.tick_params(axis='both', which='both', reset=True, direction='inout', labelsize='x-small', bottom=True, length=5, width=0.3, grid_color='grey', grid_alpha=0.5)
plot1.set_xlabel('Pressure (hPa)', size='small')
plot1.set_ylabel('Altitude (feet)', size='small')
plot1.set_title('Altitude vs. Pressure', color='blue', fontsize='medium', weight='bold', pad=12.0)
plot1.grid(visible=True, which='both', linewidth=0.3)
plot1.legend(['Interior Pressure (BMP180)', 'Predicted Pressure'], fontsize='small', loc='upper left')
plot1.set_xlim([1200,-50])

plt.tight_layout(pad=5.0)
plt.show()

In [ ]:
df2.columns

In [ ]:
%matplotlib inline

plt.figure(dpi=800, figsize=(10,10))
plot1 = plt.subplot2grid((3,1),(0,0))
plot2 = plt.subplot2grid((3,1),(1,0))
plot3 = plt.subplot2grid((3,1),(2,0))

plot1.scatter('Altitude (ft)', 'HDOP (ft)', marker='.', data=df2, color='darkgreen', alpha=0.4, linewidth=0.8)
plot1.tick_params(axis='both', which='both', reset=True, direction='inout', labelsize='x-small', bottom=True, length=5, width=0.3, grid_color='grey', grid_alpha=0.5)
plot1.set_xlabel('Altitude (ft)', size='small')
plot1.set_ylabel('HDOP (ft)', size='small')
plot1.set_title('HDOP vs. Altitude', color='blue', fontsize='medium', weight='bold', pad=12.0)
plot1.grid(visible=True, which='both', linewidth=0.3)

plot2.scatter('Speed (mph)', 'HDOP (ft)', marker='.', data=df2, color='darkgreen', alpha=0.4, linewidth=0.8)
plot2.tick_params(axis='both', which='both', reset=True, direction='inout', labelsize='x-small', bottom=True, length=5, width=0.3, grid_color='grey', grid_alpha=0.5)
plot2.set_xlabel('Speed (mph)', size='small')
plot2.set_ylabel('HDOP (ft)', size='small')
plot2.set_title('HDOP vs. Speed', color='blue', fontsize='medium', weight='bold', pad=12.0)
plot2.grid(visible=True, which='both', linewidth=0.3)

plot3.scatter('Rate (fps)', 'HDOP (ft)', marker='.', data=df2, color='orange', alpha=0.5, linewidth=0.8)
plot3.tick_params(axis='both', which='both', reset=True, direction='inout', labelsize='x-small', bottom=True, length=5, width=0.3, grid_color='grey', grid_alpha=0.5)
plot3.set_xlabel('Ascent/Descent Rate (fps)', size='small')
plot3.set_ylabel('HDOP (ft)', size='small')
plot3.set_title('HDOP vs. Rate', color='blue', fontsize='medium', weight='bold', pad=12.0)
plot3.grid(visible=True, which='both', linewidth=0.3)

plt.tight_layout(pad=2.0)
plt.ion()
plt.show()

In [ ]:
df2.columns

In [ ]:
%matplotlib inline

plt.figure(dpi=200, figsize=(8,20))
plot1 = plt.subplot2grid((2,1),(0,0))

plot1.scatter('I Pressure', 'Interior Temp (C)', marker='.', data=df2, color='darkgreen', alpha=0.4, linewidth=0.8)
plot1.tick_params(axis='both', which='both', reset=True, direction='inout', labelsize='x-small', bottom=True, length=5, width=0.3, grid_color='grey', grid_alpha=0.5)
plot1.set_xlabel('Pressure (hPa)', size='small')
plot1.set_ylabel('Temperature (C)', size='small')
plot1.set_title('Interior Temp vs. Interior Pressure', color='blue', fontsize='small', weight='bold', pad=12.0)
plot1.grid(visible=True, which='both', linewidth=0.3)

plt.tight_layout(pad=2.0)
plt.ion()
plt.show()

In [ ]:
df2.columns

In [ ]:
df2.head(5)

In [ ]:
df2['Lat_DD'] = df2['Lat'].apply(convert_lat_gps_coord)

In [ ]:
df2['Lon_DD'] = df2['Lon'].apply(convert_lon_gps_coord)

In [ ]:
df2.tail(5)

In [ ]:
# Drop last row since it's invalid
# df2.drop(df2.tail(1).index,inplace=True)

In [ ]:
df2.tail(5)

In [ ]:
mid = int(len(df2['Lon_DD'])/2)

In [ ]:
scl = [0,"rgb(150,0,90)"],[0.125,"rgb(0, 0, 200)"],[0.25,"rgb(0, 25, 255)"],\
[0.375,"rgb(0, 152, 255)"],[0.5,"rgb(44, 255, 150)"],[0.625,"rgb(151, 255, 0)"],\
[0.75,"rgb(255, 234, 0)"],[0.875,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]

In [ ]:
fig = go.Figure(go.Scattermapbox(
    name = 'Flight Path',
    mode = "markers+lines",
    lon = df2['Lon_DD'],
    lat = df2['Lat_DD'],
    hoverinfo = 'lon+lat',
    hovertemplate =
    "Longitude: %{lon}<br>" +
    "Latitude: %{lat}<br>" + 
    "%{text}",
    text = 'Altitude: ' + df2['Altitude (ft)'].astype(str) + ' (ft)<br>Speed:' + df2['Speed (mph)'].astype(str) +  '<br>Int Temp: ' + df2['Interior Temp (C)'].astype(str) + '<br>Int Pressure: ' + df2['Interior Pressure'].astype(str),
    marker = dict(
        size = 7,
        opacity = 1.0,
        symbol = 'circle',    
        reversescale = False,
        autocolorscale = False,
        colorscale = scl,
        cmin = 0,
        color = df2['Altitude (ft)'], 
        cmax = df2['Altitude (ft)'].max(),
        colorbar_title="<b>High Altitude Balloon</b><br>April 21st 2023")))

In [ ]:
fig.update_layout(
    height = 750,
    width = 1500,
    margin ={'l':10,'t':50,'b':10,'r':10},
    mapbox = {
        'center': {'lon': df2['Lon_DD'].iloc[mid], 'lat': df2['Lat_DD'].iloc[mid]},
        'style': "stamen-terrain",
        'zoom': 9.5,
        })

fig.update_layout(
        title = '<b>High Altitude Balloon - Flight Path & Altitude</b>',
        geo_scope='usa',
        geo = dict(
            scope = 'north america',
            showland = True,
            landcolor = "rgb(212, 212, 212)",
            subunitcolor = "rgb(255, 255, 255)",
            countrycolor = "rgb(255, 255, 255)",
            showlakes = True,
            lakecolor = "rgb(255, 255, 255)",
            showsubunits = True,
            showcountries = True,
            resolution = 50,
            projection = dict(
                type = 'conic conformal',
                rotation_lon = -100
            ),
            lonaxis = dict(
                showgrid = True,
                gridwidth = 0.5,
                range= [ -140.0, -55.0 ],
                dtick = 5
            ),
            lataxis = dict (
                showgrid = True,
                gridwidth = 0.5,
                range= [ 20.0, 60.0 ],
                dtick = 5
            )
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=14,
            font_family="Arial"
        ),
    )

fig.show()